In [1]:
# =====================================================================================
# ENVIRONMENT
# =====================================================================================

import snman
from snman.constants import *
from snman import osmnx_customized as oxc

PERIMETER = 'hardbruecke'
# set SAVE_TO_DEBUG = True for saving the cached network into the _debug folder
# which is automatically used in the QGIS files
SAVE_TO_DEBUG = True
INTERSECTION_TOLERANCE = 10

# set these paths according to your own setup
data_directory = 'C:/Users/lballo/polybox/Research/SNMan/SNMan Shared/data_v2/'
inputs_path = data_directory + 'inputs/'
if SAVE_TO_DEBUG:
    process_path = data_directory + 'process/' + '_debug' + '/'
else:
    process_path = data_directory + 'process/' + PERIMETER + '/'

oxc.settings.useful_tags_way = OSM_TAGS

C:\Users\lballo\AppData\Roaming\Python\Python39\site-packages\osmnx\utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


In [2]:
# =====================================================================================
# LOAD DATA
# =====================================================================================

print('Load perimeters')
perimeters = snman.io.load_perimeters(inputs_path + 'perimeters/perimeters.shp')

print('Get data from OSM server')
# At this step, simplification means only removing degree=2 edges
G = oxc.graph_from_polygon(
    # set the perimeter here
    perimeters.to_crs(4326).loc[PERIMETER]['geometry'],
    custom_filter=snman.constants.OSM_FILTER,
    simplify=True, simplify_strict=False, retain_all=True, one_edge_per_direction=False
)

print('Load manual intersections')
# polygons used to override the automatically detected intersections in some situations
given_intersections_gdf = snman.io.load_intersections(
    inputs_path + 'intersection_polygons/intersection_polygons.shp'
)

print('Prepare graph')
snman.street_graph.prepare_graph(G)

print('Convert CRS of street graph')
snman.street_graph.convert_crs(G, DEFAULT_CRS)

print('Generate lanes')
# interpreting the OSM tags into a collection of lanes on each edge
snman.space_allocation.generate_lanes(G)

print('Load sensors and assign them to edges in the raw street graph')
sensors_df = snman.io.load_sensors(inputs_path + 'sensors/sensors.csv')
snman.enrichment.match_sensors(G, sensors_df)

Load perimeters
Get data from OSM server
Load manual intersections
Prepare graph
Convert CRS of street graph
Generate lanes
Load sensors and assign them to edges in the raw street graph


In [3]:
# =====================================================================================
# ENRICH
# =====================================================================================

if 1:
    print('Identify hierarchy')
    # split the edges into hierarchy categories, such as main roads, local roads, etc.
    snman.hierarchy.add_hierarchy(G)

Identify hierarchy


In [5]:
# =====================================================================================
# CONSOLIDATE PARALLEL AND CONSECUTIVE EDGES
# =====================================================================================
if 1:
    print('Simplify edge geometries')
    snman.simplification.simplify_edge_geometries(G, 5)

for i in range(3):

    print('ITERATION', i)

    print('Detect intersections')
    intersections_gdf = snman.simplification.merge_nodes_geometric(
        G, INTERSECTION_TOLERANCE,
        given_intersections_gdf=given_intersections_gdf
    )

    print('Split through edges in intersections')
    snman.simplification.split_through_edges_in_intersections(G, intersections_gdf)

    print('Detect intersections (repeat to ensure that no points are outside of intersections)')
    intersections_gdf = snman.simplification.merge_nodes_geometric(
        G, INTERSECTION_TOLERANCE,
        given_intersections_gdf=given_intersections_gdf
    )

    print('Add layers to nodes')
    snman.simplification.add_layers_to_nodes(G)

    print('Add connections between components in intersections')
    snman.simplification.connect_components_in_intersections(G, intersections_gdf, separate_layers=True)

    print('Consolidate intersections')
    G = snman.simplification.consolidate_intersections(
        G, intersections_gdf,
        reconnect_edges=True
    )

    print('Merge consecutive edges')
    snman.merge_edges.merge_consecutive_edges(G)

    print('Merge parallel edges')
    snman.merge_edges.merge_parallel_edges(G)

    print('Update precalculated attributes')
    snman.street_graph.update_precalculated_attributes(G)

snman.simplification.simplify_edge_geometries(G, 35)
G = snman.graph_utils.keep_only_the_largest_connected_component(G, weak=True)


Simplify edge geometries
ITERATION 0
Detect intersections
Split through edges in intersections
Detect intersections (repeat to ensure that no points are outside of intersections)
Add layers to nodes
Add connections between components in intersections
Consolidate intersections
Merge consecutive edges
Merge parallel edges


C:\Users\lballo\AppData\Roaming\Python\Python39\site-packages\shapely\linear.py:208: RuntimeWarning: invalid value encountered in shortest_line
  return lib.shortest_line(a, b, **kwargs)


Update precalculated attributes
ITERATION 1
Detect intersections
Split through edges in intersections
not len(nodes) == len(split_points) + 2 == len(edge_linestrings) + 1
[(21, <POINT (2681368.723 1248100.821)>), (574, <POINT (2681397.077 1248085.578)>), (575, <POINT (2681424.323 1248070.93)>), (576, <POINT (2681449.158 1248061.914)>), (577, <POINT (2681456.397 1248074.839)>), (578, <POINT (2681474.065 1248087.794)>), (579, <POINT (2681469.026 1248122.091)>), (580, <POINT (2681468.35 1248120.514)>), (581, <POINT (2681437.958 1248133.319)>), (171, <POINT (2681397.433 1248141.923)>)]
[<POINT (2681449.158 1248061.914)> <POINT (2681424.323 1248070.93)>
 <POINT (2681456.397 1248074.839)> <POINT (2681397.077 1248085.578)>
 <POINT (2681474.065 1248087.794)> <POINT (2681468.468 1248120.789)>
 <POINT (2681437.957 1248133.319)>]
[<LINESTRING (2681368.723 1248100.821, 2681396.989 1248085.625)>, <LINESTRING (2681397.165 1248085.531, 2681397.165 1248085.531, 2681424.235 1...>, <LINESTRING (2681424.

C:\Users\lballo\AppData\Roaming\Python\Python39\site-packages\shapely\linear.py:208: RuntimeWarning: invalid value encountered in shortest_line
  return lib.shortest_line(a, b, **kwargs)


Split through edges in intersections
not len(nodes) == len(split_points) + 2 == len(edge_linestrings) + 1
[(5, <POINT (2681303.653 1248570.807)>), (649, <POINT (2681277.565 1248589.425)>), (650, <POINT (2681277.054 1248584.1)>), (651, <POINT (2681242.103 1248592.432)>), (652, <POINT (2681241.317 1248589.763)>), (8, <POINT (2681196.572 1248584.591)>)]
[<POINT (2681277.454 1248588.271)> <POINT (2681241.919 1248591.809)>]
[<LINESTRING (2681303.653 1248570.807, 2681301.151 1248577.934, 2681277.754 1...>, <LINESTRING (2681277.464 1248588.37, 2681277.445 1248588.171)>, <LINESTRING (2681277.666 1248590.481, 2681277.578 1248590.528, 2681274.143 1...>, <LINESTRING (2681241.947 1248591.905, 2681241.891 1248591.713)>, <LINESTRING (2681242.258 1248592.96, 2681201.997 1248590.306, 2681196.572 12...>]
not len(nodes) == len(split_points) + 2 == len(edge_linestrings) + 1
[(5, <POINT (2681303.653 1248570.807)>), (649, <POINT (2681316.536 1248546.647)>), (650, <POINT (2681325.581 1248534.806)>), (651, <

C:\Users\lballo\AppData\Roaming\Python\Python39\site-packages\pandas\core\arraylike.py:495: RuntimeWarning: invalid value encountered in shortest_line
  return getattr(ufunc, method)(*new_inputs, **kwargs)


not len(nodes) == len(split_points) + 2 == len(edge_linestrings) + 1
[(354, <POINT (2681516.675 1248932.005)>), (753, <POINT (2681534.523 1248967.545)>), (754, <POINT (2681543.924 1248992.918)>), (755, <POINT (2681561.227 1249018.01)>), (756, <POINT (2681586.161 1249049.341)>), (757, <POINT (2681608.011 1249091.249)>), (758, <POINT (2681624.961 1249125.205)>), (759, <POINT (2681626.766 1249124.023)>), (760, <POINT (2681651.852 1249166.934)>), (761, <POINT (2681653.65 1249165.831)>), (762, <POINT (2681673.772 1249200.64)>), (763, <POINT (2681675.536 1249199.557)>), (764, <POINT (2681700.002 1249241.547)>), (765, <POINT (2681701.798 1249240.602)>), (766, <POINT (2681740.268 1249303.828)>), (767, <POINT (2681741.938 1249302.82)>), (768, <POINT (2681759.523 1249327.329)>), (769, <POINT (2681785.504 1249367.614)>), (770, <POINT (2681804.993 1249396.829)>), (507, <POINT (2681824.594 1249419.044)>)]
[<POINT (2681561.227 1249018.01)> <POINT (2681586.161 1249049.341)>
 <POINT (2681625.081 12491

In [6]:
# =====================================================================================
# UPDATE PRE-CALCULATED ATTRIBUTES
# =====================================================================================

print('Add lane stats to edges')
# how many lanes, how wide, etc.
snman.space_allocation.generate_lane_stats(G)

print('Update OSM tags')
# to match the simplified and merged edges
snman.space_allocation.update_osm_tags(G)

if 0:
    print('Update street counts per node')
    # TODO: include in a function for updating pre-calculated attributes, adjust for directed graph
    spn = oxc.stats.count_streets_per_node(G, nodes=G.nodes)
    nx.set_node_attributes(G, values=spn, name="street_count")

Add lane stats to edges
Update OSM tags


In [7]:
# =====================================================================================
# ENRICH
# =====================================================================================

if 0:
    #TODO: use mapmatching for better performance and accuracy
    #TODO: add route direction for one-way sections
    print('Add public transport')
    pt_network = snman.io.import_geofile_to_gdf(inputs_path + "public_transit/ZVV_LINIEN_GEN_L.shp")
    snman.enrichment.match_pt(G, pt_network)

if 1:
    print('Add elevation')
    G = oxc.elevation.add_node_elevations_raster(G, inputs_path + 'ch_dhm_25/2056/ch_dhm_2056.tif', cpus=1)
    G = oxc.elevation.add_edge_grades(G, add_absolute=False)

if 0:
    print('Add traffic counts')
    source = gpd.read_file(inputs_path + 'traffic_volumes/npvm_2017_filtered.gpkg').to_crs(2056)
    source['fid'] = source.index
    # Remove links with zero traffic (otherwise they will distort the averages on the matched links)
    source = source[source['DTV_ALLE'] > 0]
    snman.enrichment.match_linestrings(G, source, [
        {'source_column': 'DTV_ALLE',   'target_column': 'adt_avg',         'agg': 'avg' },
        {'source_column': 'DTV_ALLE',   'target_column': 'adt_max',         'agg': 'max' },
        {'source_column': 'FROMNODENO', 'target_column': 'npvm_fromnodeno', 'agg': 'list'},
        {'source_column': 'TONODENO',   'target_column': 'npvm_tonodeno',   'agg': 'list'}
    ])

Add elevation


In [8]:
# =====================================================================================
# EXPORT
# =====================================================================================

if 1:
    print('Export street graph graph')
    # each street is one edge, the lanes are saved as an attribute
    snman.io.export_street_graph(G, process_path + 'street_graph_edges.gpkg', process_path + 'street_graph_nodes.gpkg')

if 1:
    print('Export lane geometries')
    # each lane has an own geometry and with as an attribute, for visualization purposes
    snman.io.export_street_graph_with_lanes(G, KEY_LANES_DESCRIPTION, process_path + 'lane_geometries.shp', scaling=2)

if 1:
    print('Save intersection geometries into a file')
    snman.io.export_gdf(intersections_gdf, process_path + 'intersections_polygons.gpkg', columns=['geometry'])

if 1:
    print('Export as OSM')
    export_osm_tags = {
            'highway', 'maxspeed',
            'lanes', 'lanes:forward', 'lanes:backward', 'lanes:both_ways', 'oneway',
            'cycleway', 'cycleway:lane', 'cycleway:left', 'cycleway:left:lane', 'cycleway:right', 'cycleway:right:lane',
            'bus:lanes:backward', 'bus:lanes:forward', 'vehicle:lanes:backward', 'vehicle:lanes:forward',
            '_connected_component'
        }

    snman.io.export_osm_xml(G, process_path + 'osm.osm', export_osm_tags, uv_tags=True, tag_all_nodes=False)

Export street graph graph
Export lane geometries
Save intersection geometries into a file
Export as OSM
